In [1]:
# univariate lstm example
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
# preparing independent and dependent features
def prepare_data(timeseries_data, look_back):
    X, y =[],[]
    for i in range(len(timeseries_data)):
        # find the end of this pattern
        end_ix = i + look_back
        # check if we are beyond the sequence
        if end_ix > len(timeseries_data)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = timeseries_data[i:end_ix], timeseries_data[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [5]:
# define input sequence
timeseries_data = [110, 120, 125, 128, 119, 123, 125, 128, 123, 127]
# choose a number of time steps
look_back = 3
# split into samples
X, y = prepare_data(timeseries_data, look_back)

In [9]:
X,y

(array([[110, 120, 125],
        [120, 125, 128],
        [125, 128, 119],
        [128, 119, 123],
        [119, 123, 125],
        [123, 125, 128],
        [125, 128, 123]]),
 array([128, 119, 123, 125, 128, 123, 127]))

In [10]:
X.shape

(7, 3)

In [11]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

## Building LSTM Model

In [15]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(look_back,n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)

Epoch 1/200
7/7 [==============================] - 2s 320ms/sample - loss: 16405.0508
Epoch 2/200
7/7 [==============================] - 0s 2ms/sample - loss: 16072.8799
Epoch 3/200
7/7 [==============================] - 0s 2ms/sample - loss: 15763.8965
Epoch 4/200
7/7 [==============================] - 0s 2ms/sample - loss: 15486.9297
Epoch 5/200
7/7 [==============================] - 0s 2ms/sample - loss: 15220.9600
Epoch 6/200
7/7 [==============================] - 0s 2ms/sample - loss: 14981.6533
Epoch 7/200
7/7 [==============================] - 0s 2ms/sample - loss: 14780.4062
Epoch 8/200
7/7 [==============================] - 0s 2ms/sample - loss: 14582.0986
Epoch 9/200
7/7 [==============================] - 0s 2ms/sample - loss: 14387.3232
Epoch 10/200
7/7 [==============================] - 0s 3ms/sample - loss: 14188.3818
Epoch 11/200
7/7 [==============================] - 0s 2ms/sample - loss: 13981.0020
Epoch 12/200
7/7 [==============================] - 0s 2ms/sample - loss

## Predicting For the next 10 data

In [18]:
np.array(timeseries_data[-3:])

array([128, 123, 127])

In [25]:
# demonstrate prediction for next 10 days
x_input = np.array([128, 123, 127])
temp_input=list(x_input)
lst_output=[]
i=0
while(i<10):
    
    if(len(temp_input)>3):
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        #print(x_input)
        x_input = x_input.reshape((1, look_back, n_features))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        print("{} day output {}".format(i,yhat))
        temp_input.append(yhat[0][0])
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.append(yhat[0][0])
        i=i+1
    else:
        x_input = x_input.reshape((1, look_back, n_features))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.append(yhat[0][0])
        lst_output.append(yhat[0][0])
        i=i+1
    

print(lst_output)

[128.56685]
1 day input [123.         127.         128.56684875]
1 day output [[126.9623]]
2 day input [127.         128.56684875 126.96230316]
2 day output [[129.66348]]
3 day input [128.56685 126.9623  129.66348]
3 day output [[130.48398]]
4 day input [126.9623  129.66348 130.48398]
4 day output [[130.43983]]
5 day input [129.66348 130.48398 130.43983]
5 day output [[132.34569]]
6 day input [130.48398 130.43983 132.34569]
6 day output [[133.08978]]
7 day input [130.43983 132.34569 133.08978]
7 day output [[133.74586]]
8 day input [132.34569 133.08978 133.74586]
8 day output [[135.2413]]
9 day input [133.08978 133.74586 135.2413 ]
9 day output [[136.10092]]
[128.56685, 126.9623, 129.66348, 130.48398, 130.43983, 132.34569, 133.08978, 133.74586, 135.2413, 136.10092]


In [26]:
timeseries_data+lst_output

[110,
 120,
 125,
 128,
 119,
 123,
 125,
 128,
 123,
 127,
 128.56685,
 126.9623,
 129.66348,
 130.48398,
 130.43983,
 132.34569,
 133.08978,
 133.74586,
 135.2413,
 136.10092]